In [8]:
import os

import cv2 as cv
import numpy as np

In [9]:
input_folder_path = "tlights"
img_names = os.listdir(input_folder_path)
result_folder = "result"

if not os.path.exists(result_folder):
    os.makedirs(result_folder)


def get_bgr_image(img_name: str, folder_path: str = input_folder_path) -> np.ndarray[np.uint8]:
    return cv.imread(os.path.join(folder_path, img_name))


def convert_bgr_to_hsv(img_bgr: np.ndarray[np.uint8]) -> np.ndarray[np.uint8]:
    return cv.cvtColor(img_bgr, cv.COLOR_BGR2HSV)


def write_filenames_to_txt(result: list[str], color: str) -> None:
    with open(f"{result_folder}/{color}_tlights.txt", "w") as f:
        for line in result:
            f.write(f"{line}\n")

In [10]:
# Диапазоны для красного цвета
lower_red_1 = np.array([0, 100, 100])
upper_red_1 = np.array([15, 255, 255])
lower_red_2 = np.array([170, 100, 100])
upper_red_2 = np.array([180, 255, 255])

# Диапазон для желтого цвета
lower_yellow = np.array([15, 100, 100])
upper_yellow = np.array([35, 255, 255])

# Диапазон для зеленого цвета
lower_green = np.array([60, 100, 100])
upper_green = np.array([90, 255, 255])

In [11]:
def get_mask(
    img_hsv: np.ndarray[np.uint8], 
    lower_border: np.ndarray[np.uint8], 
    upper_border: np.ndarray[np.uint8]
) -> np.ndarray[np.uint8]:
    h, s, v = img_hsv[:, :, 0], img_hsv[:, :, 1], img_hsv[:, :, 2]

    mask = np.where(
        (h >= lower_border[0]) & (h <= upper_border[0])
        & (s >= lower_border[1]) & (s <= upper_border[1])
        & (v >= lower_border[2]) & (v <= upper_border[2]),
        1, 0,
    ).astype(np.uint8)
    return mask

In [12]:
def get_colors_probability(img_hsv: np.ndarray[np.uint8]) -> tuple[float]:
    mask_yellow = get_mask(img_hsv=img_hsv, lower_border=lower_yellow, upper_border=upper_yellow)
    mask_red_1 = get_mask(img_hsv=img_hsv, lower_border=lower_red_1, upper_border=upper_red_1)
    mask_red_2 = get_mask(img_hsv=img_hsv, lower_border=lower_red_2, upper_border=upper_red_2)
    mask_green = get_mask(img_hsv=img_hsv, lower_border=lower_green, upper_border=upper_green)

    count_yellow_color = mask_yellow.sum()
    count_red_color = mask_red_1.sum() + mask_red_2.sum()
    count_green_color = mask_green.sum()
    count_all_colors = count_yellow_color + count_red_color + count_green_color

    prob_yellow_color = count_yellow_color / count_all_colors
    prob_red_color = count_red_color / count_all_colors
    prob_green_color = count_green_color / count_all_colors

    return prob_yellow_color, prob_red_color, prob_green_color

In [13]:
files_yellow_tlights = []
files_red_tlights = []
files_green_tlights = []
files_undefined_tlights = []

for img_name in img_names:
    img_bgr = get_bgr_image(img_name=img_name)
    img_hsv = convert_bgr_to_hsv(img_bgr=img_bgr)

    prob_yellow_color, prob_red_color, prob_green_color = get_colors_probability(img_hsv=img_hsv)

    if prob_yellow_color > 0.65:
        files_yellow_tlights.append(img_name)
    elif prob_red_color > 0.65:
        files_red_tlights.append(img_name)
    elif prob_green_color > 0.65:
        files_green_tlights.append(img_name)
    else:
        files_undefined_tlights.append(img_name)


In [14]:
write_filenames_to_txt(result=files_yellow_tlights, color="yellow")
write_filenames_to_txt(result=files_red_tlights, color="red")
write_filenames_to_txt(result=files_green_tlights, color="green")
write_filenames_to_txt(result=files_undefined_tlights, color="undefined")